In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import os
input_fname = 'test_IOTA.in'
np.set_printoptions(precision=3)

# Prepare Beam and Lattice

In [2]:
beam, lattice0 = impact.readInputFile(input_fname)

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [3]:
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=10
beam.current=0.0
dist = beam.distribution
dist.distribution_type = 'Waterbag'
lattice = [impact.getElem('loop'),
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl')]
lattice[0].nturn = 1
lattice[1].file_id = 1001
lattice[3].file_id = 1002
lattice[5].file_id = 1003


ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

In [4]:
dist.emitx = dist.emitx
dist.emity = dist.emity

# 1. Drift

In [5]:
elemF = impact.getElem('drift')
elemF.length = 1.1
elemF.n_sckick = 20
elemF.n_map = 2

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

input error <- sum(beam.multi_charge.n_particles) not qual to beam.n_particles
  ... enforcing  beam.multi_charge.n_particles[0] to beam.n_particles
input error <- beam.multi_charge.current[0] not qual to beam.current
  ... enforcing  beam.multi_charge.current[0] to beam.current


0

In [6]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.084e-19  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  6.437e-20  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-1.626e-19  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]]


# 2. Quad

In [7]:
elemF = impact.getElem('quad')
elemF.length = 0.02
elemF.n_sckick = 20
elemF.n_map = 1
elemF.B1 = 15.0

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [8]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00 -1.735e-18 -6.505e-19  2.385e-18  0.000e+00  0.000e+00]
 [ 1.735e-18  0.000e+00  6.505e-18 -5.638e-18  0.000e+00  0.000e+00]
 [ 1.735e-18  3.795e-19  6.505e-18 -3.903e-18  0.000e+00  0.000e+00]
 [ 1.735e-18 -1.301e-18 -3.686e-18  3.903e-18  0.000e+00  0.000e+00]
 [ 8.674e-19 -6.505e-19 -4.879e-19  4.770e-18  0.000e+00  0.000e+00]
 [ 0.000e+00 -8.674e-19  3.388e-20 -3.253e-18  0.000e+00  0.000e+00]
 [ 2.168e-19  1.301e-18  3.469e-18 -4.120e-18  0.000e+00  0.000e+00]
 [ 0.000e+00 -6.505e-19  2.385e-18 -4.337e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -7.806e-18  3.036e-18  0.000e+00  0.000e+00]
 [ 1.301e-18  4.337e-18 -4.337e-18  2.819e-18  0.000e+00  0.000e+00]]


# 3. Sextupole

In [9]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'multipole_thin':
        break
lattice0[i]


 KL_dipole: 0.0
   KL_quad: 0.0
   KL_sext: 0.15
    KL_oct: 0.0
   KL_deca: 0.0
 KL_dodeca: 0.0
      type: 'multipole_thin'

In [10]:
elemF = lattice0[i]

elemB = copy(elemF)
elemF.KL_sext = 10.0
elemB.KL_sext =-elemF.KL_sext

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [11]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData1[:,:6]-pData0[:,:6])

[[ 0.000e+00 -9.800e-05  0.000e+00  1.060e-05  0.000e+00  0.000e+00]
 [ 0.000e+00 -7.642e-06  0.000e+00  6.272e-05  0.000e+00  0.000e+00]
 [ 0.000e+00  1.752e-06  0.000e+00 -2.929e-05  0.000e+00  0.000e+00]
 [ 0.000e+00 -5.593e-06  0.000e+00 -2.780e-05  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.270e-06  0.000e+00 -1.888e-06  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.490e-05  0.000e+00 -4.161e-07  0.000e+00  0.000e+00]
 [ 0.000e+00  2.032e-05  0.000e+00 -1.029e-05  0.000e+00  0.000e+00]
 [ 0.000e+00  4.868e-06  0.000e+00 -3.036e-06  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.622e-05  0.000e+00 -4.963e-05  0.000e+00  0.000e+00]
 [ 0.000e+00 -1.328e-05  0.000e+00 -2.701e-05  0.000e+00  0.000e+00]]


In [12]:
print(pData2[:,:6]-pData0[:,:6])

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# 4. Dipole

In [13]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'dipole':
        break
lattice0[i]


                   length: 0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: 1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.058 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: 0.5 [1]
                     type: 'dipole'

### 4.1 without fringe field

##### negative length 

In [14]:
elemF = copy(lattice0[i])
elemF.fringe_field_integration = 0.0

elemB = copy(elemF)
elemB.length = -elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [15]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 2.892e-13 -7.155e-13  0.000e+00  0.000e+00  2.397e+00  0.000e+00]
 [-1.096e-12  2.713e-12  0.000e+00  0.000e+00 -1.465e+00  0.000e+00]
 [-1.396e-12  3.455e-12  0.000e+00  0.000e+00  1.546e+00  0.000e+00]
 [ 1.058e-12 -2.619e-12  0.000e+00  0.000e+00 -9.348e-01  0.000e+00]
 [ 1.994e-12 -4.933e-12  1.626e-19  0.000e+00 -1.084e+00  0.000e+00]
 [ 3.152e-13 -7.798e-13 -1.423e-19  0.000e+00  1.467e+00  0.000e+00]
 [ 9.008e-13 -2.229e-12  0.000e+00  0.000e+00  1.056e+00  0.000e+00]
 [-1.160e-12  2.871e-12  0.000e+00  0.000e+00  6.892e-01  0.000e+00]
 [ 1.907e-12 -4.718e-12  0.000e+00  0.000e+00 -1.816e+00  0.000e+00]
 [-9.624e-13  2.381e-12  0.000e+00  0.000e+00 -7.756e-01  0.000e+00]]


### negative length and bending angle

In [16]:
elemB.bending_angle = -elemF.bending_angle
impact.writeInputFile(beam,lattice);
impact.run()

0

In [17]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-9.541e-18  1.735e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.735e-18 -3.903e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-1.735e-18 -8.132e-19  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.735e-18 -1.301e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.843e-18  2.819e-18  1.626e-19  0.000e+00  0.000e+00  0.000e+00]
 [-1.735e-18  3.036e-18 -1.423e-19  0.000e+00  0.000e+00  0.000e+00]
 [-2.494e-18 -4.120e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-9.758e-19 -2.168e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-1.735e-18 -1.518e-18  0.000e+00  0.000e+00  5.684e-14  0.000e+00]
 [ 1.301e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]]


### 4.2 with fringe field

In [18]:
elemF = copy(lattice0[i])

elemB = copy(elemF)
elemB.length = -elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [19]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 2.892e-13 -7.155e-13  3.494e-05 -1.728e-05  2.397e+00  0.000e+00]
 [-1.096e-12  2.713e-12  7.055e-05 -3.712e-04 -1.465e+00  0.000e+00]
 [-1.396e-12  3.455e-12  6.605e-05 -2.950e-04  1.546e+00  0.000e+00]
 [ 1.058e-12 -2.619e-12 -4.038e-07  1.792e-04 -9.348e-01  0.000e+00]
 [ 1.994e-12 -4.933e-12  6.313e-05 -5.145e-05 -1.084e+00  0.000e+00]
 [ 3.152e-13 -7.798e-13 -8.114e-05  1.037e-04  1.467e+00  0.000e+00]
 [ 9.008e-13 -2.229e-12  7.743e-05 -3.469e-04  1.056e+00  0.000e+00]
 [-1.160e-12  2.871e-12  7.252e-05 -2.166e-04  6.892e-01  0.000e+00]
 [ 1.907e-12 -4.718e-12 -8.541e-05  3.163e-04 -1.816e+00  0.000e+00]
 [-9.624e-13  2.381e-12 -3.811e-05  2.033e-04 -7.756e-01  0.000e+00]]


### negative length, bending angle and FINT

###### MaryLie_order = 1

In [20]:
elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
impact.writeInputFile(beam,lattice);
impact.run()
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-9.541e-18  1.735e-18  1.084e-19 -5.529e-18  0.000e+00  0.000e+00]
 [ 1.735e-18 -3.903e-18  6.549e-17 -1.398e-15  0.000e+00  0.000e+00]
 [-1.735e-18 -8.132e-19 -3.665e-17  7.654e-16  0.000e+00  0.000e+00]
 [ 1.735e-18 -1.301e-18 -4.901e-17  1.046e-15  0.000e+00  0.000e+00]
 [ 1.843e-18  2.819e-18 -1.241e-17  2.639e-16  0.000e+00  0.000e+00]
 [-1.735e-18  3.036e-18  3.659e-19 -8.023e-18  0.000e+00  0.000e+00]
 [-2.494e-18 -4.120e-18  2.385e-17 -4.723e-16  0.000e+00  0.000e+00]
 [-9.758e-19 -2.168e-18  2.645e-17 -5.636e-16  0.000e+00  0.000e+00]
 [-1.735e-18 -1.518e-18 -9.541e-18  2.082e-16  5.684e-14  0.000e+00]
 [ 1.301e-18  0.000e+00 -1.388e-17  2.982e-16  0.000e+00  0.000e+00]]


###### MaryLie_order = 2

elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
os.system('xmain_dipoleF2')
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

###### MaryLie_order = 3

elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
impact.writeInputFile(beam,lattice);
os.system('xmain_dipoleF3')
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

# 5. Nonlinear Insert

In [21]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'nonlinear_insert':
        break
lattice0[i]


             length: 1.8 [m]
           n_sckick: 96 [1]
              n_map: 5 [1]
         strength_t: 0.4
 transverse_scale_c: 0.01
        pipe_radius: 10.0 [m]
       tune_advance: 0.3034496449165134
               type: 'nonlinear_insert'

In [22]:
elemF = lattice0[i]

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [23]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-2.949e-17  2.429e-17  2.526e-17 -2.320e-17  1.421e-14  0.000e+00]
 [ 2.342e-17 -1.778e-17 -4.770e-18  8.240e-18  0.000e+00  0.000e+00]
 [ 3.253e-18  1.420e-17  4.770e-18  0.000e+00  0.000e+00  0.000e+00]
 [ 1.952e-17 -5.204e-18  2.624e-17 -2.732e-17  0.000e+00  0.000e+00]
 [ 3.144e-18 -1.236e-17  5.627e-17 -5.378e-17  0.000e+00  0.000e+00]
 [-9.541e-18  1.561e-17 -5.462e-17  5.356e-17  0.000e+00  0.000e+00]
 [ 1.084e-17  9.541e-18  1.778e-17 -4.120e-18 -5.684e-14  0.000e+00]
 [ 5.096e-18  8.890e-18  3.123e-17 -2.488e-17  0.000e+00  0.000e+00]
 [ 1.301e-17 -2.060e-17 -2.233e-17  1.258e-17  0.000e+00  0.000e+00]
 [ 1.691e-17 -1.084e-17 -1.735e-18 -3.036e-18  0.000e+00  0.000e+00]]


# 6. Linear matrix map (toy model compoenent)

In [24]:
elemF = impact.getElem('linear_matrix_map')
elemF.nonlinear_insert_length = 1.8
elemF.nonlinear_insert_tuneAdvance = 0.3034496449165134
elemF.tune_advance = 0.22

elemB = copy(elemF)
elemB.tune_advance =-elemF.tune_advance
elemB.nonlinear_insert_tuneAdvance =-elemF.nonlinear_insert_tuneAdvance
elemB.nonlinear_insert_length =-elemF.nonlinear_insert_length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [25]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 8.674e-19 -1.735e-18 -8.403e-19 -4.337e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  1.735e-18 -1.735e-18  0.000e+00  0.000e+00  0.000e+00]
 [ 1.735e-18  1.084e-19 -1.301e-18  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  1.301e-18  6.505e-19  0.000e+00  0.000e+00  0.000e+00]
 [-3.253e-19 -4.337e-19  2.711e-19 -4.337e-19  0.000e+00  0.000e+00]
 [ 0.000e+00 -4.337e-19 -3.185e-19  4.337e-19  0.000e+00  0.000e+00]
 [ 7.589e-19  1.301e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [-1.626e-19  2.168e-19 -2.168e-19  1.626e-19  0.000e+00  0.000e+00]
 [-1.735e-18  0.000e+00  0.000e+00 -4.337e-19  0.000e+00  0.000e+00]
 [-1.735e-18  1.301e-18 -6.505e-19 -8.674e-19  0.000e+00  0.000e+00]]


# 7. ToyModel

In [26]:
elemF = impact.getElem('nonlinear_insert')
elemF.tune_advance = 0.3034496449165134
elemF.length = 1.8
elemF.n_sckick = 100
elemF.n_map = 10
elemF.strength_t = 0.4
elemF.transverse_scale_c = 0.01

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

In [27]:
elemF1 = impact.getElem('linear_matrix_map')
elemF1.nonlinear_insert_length = 1.8
elemF1.nonlinear_insert_tuneAdvance = 0.3034496449165134
elemF1.tune_advance = 0.0

elemB1 = copy(elemF1)
elemB1.tune_advance =-elemB1.tune_advance
elemB1.nonlinear_insert_tuneAdvance =-elemB1.nonlinear_insert_tuneAdvance
elemB1.nonlinear_insert_length =-elemB1.nonlinear_insert_length

lattice.insert(3,elemF1)
lattice.insert(5,elemB1)

In [28]:
for i in range(len(lattice)):
    print(lattice[i].type)

loop
write_raw_ptcl
nonlinear_insert
linear_matrix_map
write_raw_ptcl
linear_matrix_map
nonlinear_insert
write_raw_ptcl


In [29]:
impact.writeInputFile(beam,lattice);
impact.run()

0

In [30]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-4.597e-17  4.857e-17  5.931e-17 -6.104e-17  0.000e+00  0.000e+00]
 [ 2.515e-17 -1.865e-17  0.000e+00  9.541e-18  0.000e+00  0.000e+00]
 [ 1.735e-18  1.480e-17  1.106e-17 -1.301e-18  0.000e+00  0.000e+00]
 [ 1.605e-17 -1.800e-17  2.667e-17 -3.426e-17  0.000e+00  0.000e+00]
 [ 1.843e-18 -1.323e-17  4.467e-17 -4.055e-17  0.000e+00  0.000e+00]
 [-1.908e-17  2.060e-17 -4.984e-17  4.207e-17  0.000e+00  0.000e+00]
 [ 9.433e-18  9.541e-18  1.475e-17 -6.722e-18  0.000e+00  0.000e+00]
 [ 2.060e-18  1.258e-17  3.404e-17 -2.933e-17  0.000e+00  0.000e+00]
 [ 2.082e-17 -2.450e-17 -2.559e-17  1.583e-17  0.000e+00  0.000e+00]
 [ 2.038e-17 -1.518e-17  6.505e-19 -5.638e-18  0.000e+00  0.000e+00]]
